In [ ]:
!nvidia-smi

Tue Dec  6 02:29:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import keras
import numpy as np
import cv2
import os
import random
import pandas as pd
import csv
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
from os import path
import keras.backend as k

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,zoom_range=0.05,rotation_range=360,width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
if path.exists('/content/drive/MyDrive/BioProject/models1') == False:
  os.mkdir('/content/drive/MyDrive/BioProject/models1')
full_name='Concatinate'
num_classes=3 
input_tensor=Input(shape=(256,256,3))

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/BioProject/All_data/val_data.csv")
k.clear_session()

In [ ]:
model = Sequential()
model1 = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
features1 = model1.output
model2 = ResNet50V2(weights='imagenet', include_top=False, input_tensor=input_tensor)
features2 = model2.output
concatenated=keras.layers.concatenate([features1,features2])
new_layer = keras.layers.Conv2D(512, (1, 1),padding='same')(concatenated)
new_layer1 = keras.layers.Conv2D(512, (1, 1),padding='same')(new_layer)
flatten_layer = Flatten()(new_layer1)
dropout = Dropout(0.5)(flatten_layer) 
final_output =Dense(num_classes, activation='softmax',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dropout)
model = Model(inputs=input_tensor, outputs=final_output)
model.summary()

94668760/94668760 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [ ]:
for layer in model.layers:
  layer.trainable = True
model.compile(optimizer=optimizers.Nadam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
filepath="/content/drive/MyDrive/BioProject/models1/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name 
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:

for trainset in range(1,9): 
  train_df =pd.read_csv("/content/drive/MyDrive/BioProject/All_data/Train_data/train{}.csv".format(trainset)) 
  train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(256, 256),
        batch_size=20,
        class_mode='categorical',shuffle=True)
  validation_generator = test_datagen.flow_from_dataframe(
          dataframe=test_df,
          directory='/content/drive/MyDrive/BioProject/All_data/All',
          x_col="filename",
          y_col="class",
          target_size=(256, 256),
          batch_size=20,
          class_mode='categorical',shuffle=True)
  model.fit_generator(train_generator, epochs=20,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) 

Found 1433 validated image filenames belonging to 3 classes.
Found 845 validated image filenames belonging to 3 classes.


<ipython-input-9-7addba0b9c5c>:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=20,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list)


Epoch 1/20
72/72 [==============================] - 658s 9s/step - loss: 0.6926 - accuracy: 0.7174 - val_loss: 0.4367 - val_accuracy: 0.8189
Epoch 2/20
72/72 [==============================] - 66s 911ms/step - loss: 0.5361 - accuracy: 0.7886 - val_loss: 0.3698 - val_accuracy: 0.8651
Epoch 3/20
72/72 [==============================] - 65s 889ms/step - loss: 0.4421 - accuracy: 0.8367 - val_loss: 0.4088 - val_accuracy: 0.8568
Epoch 4/20
72/72 [==============================] - 66s 910ms/step - loss: 0.3745 - accuracy: 0.8625 - val_loss: 0.7110 - val_accuracy: 0.6627
Epoch 5/20
72/72 [==============================] - 66s 908ms/step - loss: 0.3361 - accuracy: 0.8737 - val_loss: 0.6922 - val_accuracy: 0.7314
Epoch 6/20
72/72 [==============================] - 66s 908ms/step - loss: 0.2733 - accuracy: 0.9009 - val_loss: 0.3857 - val_accuracy: 0.8580
Epoch 7/20
72/72 [==============================] - 69s 952ms/step - loss: 0.2633 - accuracy: 0.9037 - val_loss: 0.3502 - val_accuracy: 0.8923
E

In [ ]:
reports={}
test_df = pd.read_csv("/content/drive/MyDrive/BioProject/all_test.csv") 
validation_generator = test_datagen.flow_from_dataframe( 
        dataframe=test_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(256, 256),
        batch_size=20,
        class_mode='categorical',shuffle=True)

Found 7334 validated image filenames belonging to 3 classes.


In [ ]:
trained_models=[]
for r,d,f in os.walk('/content/drive/MyDrive/BioProject/models1'):
  for file in f:
    trained_models.append(os.path.join(r,file))

In [ ]:
trained_models

['/content/drive/MyDrive/BioProject/models1/Concatinate-03-0.9337.hdf5']

In [ ]:
for trained_model in trained_models:
  k.clear_session()
  net=keras.models.load_model(trained_model)
  covid_label= validation_generator.class_indices['COVID-19']
  pneu_label= validation_generator.class_indices['pneumonia']
  normal_label= validation_generator.class_indices['normal'] 
  tp=0 
  fp=0 
  anum=0 
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 
  wrong_covid_pneu=0   
  wrong_pneu_covid=0   
  wrong_pneu_normal=0 
  wrong_normal_pneu=0 
  wrong_normal_covid=0  
  ################
  for num,img_name in enumerate(validation_generator.filenames): 
    gt_ind=validation_generator.classes[num] 
    img=cv2.resize(cv2.imread('/content/drive/MyDrive/BioProject/All_data/All/'+img_name),(256,256)) 
    img=img.astype('float32') / 255.0 
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) 
    anum+=1 

    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1

    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1
  reports[trained_model]=[correct_covid,wrong_covid,not_detected_covid,
                   correct_pneu,wrong_pneu,not_detected_pneu,
                   correct_normal,wrong_normal,not_detected_normal,
                   covid_num,pneu_num,normal_num,tp,fp,wrong_covid_normal,wrong_covid_pneu,
                   wrong_pneu_covid,wrong_pneu_normal,wrong_normal_pneu,wrong_normal_covid]
  print(trained_model)
  print('correct_covid: ',correct_covid/covid_num,'%')
  print('wrong_covid: ',wrong_covid)
  print('not_detected_covid: ',not_detected_covid/covid_num,'%')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step
/content/drive/MyDrive/BioProject/models1/Concatinate-03-0.9337.hdf5
correct_covid:  0.9777777777777777 %
wrong_covid:  36
not_detected_covid:  0.022222222222222223 %


In [ ]:
print(reports)

{'/content/drive/MyDrive/BioProject/models1/Concatinate-03-0.9337.hdf5': [44, 36, 1, 2243, 369, 177, 4479, 163, 390, 45, 2420, 4869, 6766, 568, 1, 0, 15, 162, 369, 21]}


In [ ]:
#new
print(correct_covid/covid_num)
print(correct_pneu/pneu_num)
print(correct_normal/normal_num)

0.9777777777777777
0.9268595041322314
0.9199014171287738


In [ ]:
temp = 'concat'

In [ ]:
temp = 'concat_new'

In [ ]:
#previous Concatinated Results
print(reports)

{'t/drive/MyDrive/BioProject/models/concatenate-20-0.9325.hdf5': [40, 16, 5, 2236, 313, 184, 4548, 181, 321, 45, 2420, 4869, 6824, 510, 2, 3, 5, 179, 310, 11]}


In [ ]:
with open('/content/drive/MyDrive/results/{}-fold{}.csv'.format(temp,1),newline='', mode='w') as csvfile: 
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['correct_covid','wrong_covid','not_detected_covid',
                          'correct_pneu','wrong_pneu','not_detected_pneu',
                          'correct_normal','wrong_normal','not_detected_normal',
                          'covid_num','pneu_num','normal_num','tp','fp','wrong_covid_normal','wrong_covid_pneu',
                  'wrong_pneu_covid','wrong_pneu_normal','wrong_normal_pneu','wrong_normal_covid'])
     for key in reports.keys():
         row=[key]
         csv_writer.writerow(reports[key])

In [ ]:
#testing with some random covid possitive image
k.clear_session()
net=keras.models.load_model('/content/drive/MyDrive/BioProject/models/concatenate-20-0.9325.hdf5') 
covid_label= validation_generator.class_indices['COVID-19'] 
pneu_label= validation_generator.class_indices['pneumonia']
normal_label= validation_generator.class_indices['normal']  

img=cv2.resize(cv2.imread('/content/drive/MyDrive/BioProject/All_data/All/'+'1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2b.png'),(256,256)) 
img =img.astype('float32') / 255.0
pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0])
if (pred_ind == 0):
  print('The image is covid positive')
elif (pred_ind == 1):
  print('The image is Normal')
elif (pred_ind == 2):
  print('The image is Pnemonia Positive')
print(validation_generator.class_indices)

1/1 [==============================] - 1s 1s/step
The image is covid positive
{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
